In [1]:
import torch
from torchvision import datasets, transforms

tensor_transform = transforms.Compose([
    transforms.ToTensor()
])

batch_size = 512
train_dataset = datasets.MNIST(root = "/home/zhh/data",
									train = True,
									download = True,
									transform = tensor_transform)
test_dataset = datasets.MNIST(root = "/home/zhh/data",
									train = False,
									download = True,
									transform = tensor_transform)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
							   batch_size = batch_size,
								 shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
							   batch_size = batch_size,
								 shuffle = False)

In [2]:
class Avger(list):
    def __str__(self):
        return f'{sum(self) / len(self):.4f}' if len(self) > 0 else 'N/A'
    
from tqdm.notebook import tqdm,trange
import torchvision.utils as vutils
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class BatchPrepareBase:
    def process(self, x):
        raise NotImplementedError

In [4]:
import os
import torchvision.utils as vutils
from tqdm import trange

class SamplerBase:
    def __init__(self):
        os.makedirs('samples', exist_ok=True)
        os.makedirs('checkpoints', exist_ok=True)
        
    def calc(self, model, num):
        raise NotADirectoryError()
    
    @torch.no_grad()
    def sample(self, model, num, desc):
        x = self.calc(model, num)
        grid = vutils.make_grid(x, nrow=8)
        vutils.save_image(grid, f'samples/{desc}.png')
        torch.save(model.state_dict(), f'checkpoints/{desc}.pth')
        

In [5]:
class SanityVAE_BatchPrep(BatchPrepareBase):
    def process(self, x):
        return x.cuda()
        
class NoiseVAE_BatchPrep(BatchPrepareBase):
    def process(self, x):
        noise = torch.randn_like(x)
        return ((x + noise)/2).cuda()
        

In [6]:
class VAE_Sampler(SamplerBase):

    def calc(self,model,num):
        return model.decode(torch.randn(num, model.latent_dim).cuda())
    
    @torch.no_grad()
    def sample(self, model, num, desc):
        x_data = next(iter(test_loader))[0][:num].cuda()
        
        # generation
        x = self.calc(model, num)
        grid = vutils.make_grid(x, nrow=8)
        vutils.save_image(grid, f'samples/{desc}_gen.png')
        
        # reconstruction
        
        mu, logvar = model.encode(x_data)
        x_recon = model.decode(mu+torch.randn_like(mu)*torch.exp(logvar/2))
        grid = vutils.make_grid(x_recon, nrow=8)
        vutils.save_image(grid, f'samples/{desc}_recon.png')
        grid = vutils.make_grid(x_data, nrow=8)
        vutils.save_image(grid, f'samples/{desc}_data.png')
        
        torch.save(model.state_dict(), f'checkpoints/{desc}.pth')
        

In [7]:
from tqdm import tqdm
class VAE_trainer:
    def __init__(self, model, epochs,lr,desc,preparer:BatchPrepareBase,sampler:SamplerBase, sample_ep=4):
        self.model = model
        self.epochs = epochs
        self.sample_ep = sample_ep
        self.lr = lr
        self.preparer = preparer
        self.sampler = sampler
        self.desc = desc
        
    def run(self):
        opt = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        for epoch in range(self.epochs):
            # opt.param_groups[0]['lr'] = self.lr * (1-epoch/self.epochs)
            self.model.train()
            train_loss = Avger(); rec_loss = Avger(); kl_loss = Avger()
            
            with tqdm(train_loader) as bar:
                for x,_ in bar:
                    inputs = self.preparer.process(x)
                    loss, rec, kl = self.model(inputs)
                    
                    opt.zero_grad()
                    loss.backward()
                    opt.step()
                    train_loss.append(loss.item()); rec_loss.append(rec.item()); kl_loss.append(kl.item())
                    bar.set_description(f'Epoch {epoch+1}/{self.epochs}, loss={train_loss}, rec={rec_loss}, kl={kl_loss}')
                    # break
                    
            if epoch == 0 or (epoch + 1) % self.sample_ep == 0 or epoch == self.epochs - 1:
                self.model.eval()
                self.sampler.sample(self.model, 64, f'ep{epoch+1}_{self.desc}')
                print(f'Epoch {epoch+1}, sample saved')

# VAE sanity

In [8]:
from vae import VAE

In [9]:
model = VAE().cuda()
print('number of parameters:', sum(p.numel() for p in model.parameters()))
trainer = VAE_trainer(model, 50, 1e-3, 'sanity', SanityVAE_BatchPrep(), VAE_Sampler())
trainer.run()

number of parameters: 944600


Epoch 1/50, loss=53.3275, rec=52.7250, kl=0.6025: 100%|██████████| 118/118 [00:01<00:00, 66.77it/s]


Epoch 1, sample saved


Epoch 4/50, loss=36.3023, rec=30.3727, kl=5.9295: 100%|██████████| 118/118 [00:01<00:00, 69.59it/s]


Epoch 4, sample saved


...

Epoch 44, sample saved


Epoch 45/50, loss=30.2205, rec=22.5707, kl=7.6498: 100%|██████████| 118/118 [00:01<00:00, 71.93it/s]
Epoch 46/50, loss=30.2165, rec=22.5380, kl=7.6784: 100%|██████████| 118/118 [00:01<00:00, 68.97it/s]
Epoch 47/50, loss=30.1951, rec=22.5050, kl=7.6901: 100%|██████████| 118/118 [00:01<00:00, 73.72it/s]
Epoch 48/50, loss=30.1453, rec=22.4502, kl=7.6951: 100%|██████████| 118/118 [00:01<00:00, 73.10it/s]


Epoch 48, sample saved


Epoch 49/50, loss=30.1244, rec=22.4212, kl=7.7032: 100%|██████████| 118/118 [00:01<00:00, 59.05it/s]
Epoch 50/50, loss=30.0479, rec=22.3436, kl=7.7044: 100%|██████████| 118/118 [00:01<00:00, 64.30it/s]


Epoch 50, sample saved


# VAE task

In [10]:
model = VAE().cuda()
print('number of parameters:', sum(p.numel() for p in model.parameters()))
trainer = VAE_trainer(model, 50, 1e-3, 'sanity', NoiseVAE_BatchPrep(), VAE_Sampler())
trainer.run()

number of parameters: 944600


Epoch 1/50, loss=209.6225, rec=209.5991, kl=0.0234: 100%|██████████| 118/118 [00:02<00:00, 41.22it/s]


Epoch 1, sample saved


Epoch 2/50, loss=209.0398, rec=208.6465, kl=0.3933: 100%|██████████| 118/118 [00:02<00:00, 44.75it/s]
Epoch 3/50, loss=208.7570, rec=208.1151, kl=0.6419: 100%|██████████| 118/118 [00:02<00:00, 50.63it/s]
Epoch 4/50, loss=208.5560, rec=207.8448, kl=0.7112: 100%|██████████| 118/118 [00:02<00:00, 42.76it/s]


Epoch 4, sample saved


...

Epoch 44, sample saved


Epoch 45/50, loss=207.5945, rec=205.3416, kl=2.2529: 100%|██████████| 118/118 [00:02<00:00, 45.01it/s]
Epoch 46/50, loss=207.6351, rec=205.3824, kl=2.2527: 100%|██████████| 118/118 [00:02<00:00, 47.60it/s]
Epoch 47/50, loss=207.5696, rec=205.3127, kl=2.2569: 100%|██████████| 118/118 [00:02<00:00, 41.47it/s]
Epoch 48/50, loss=207.5538, rec=205.2753, kl=2.2785: 100%|██████████| 118/118 [00:02<00:00, 51.19it/s]


Epoch 48, sample saved


Epoch 49/50, loss=207.6184, rec=205.3265, kl=2.2919: 100%|██████████| 118/118 [00:02<00:00, 52.04it/s]
Epoch 50/50, loss=207.4788, rec=205.1935, kl=2.2852: 100%|██████████| 118/118 [00:02<00:00, 46.93it/s]


Epoch 50, sample saved
